# Manufactured solutions for Elasticity

This is a notebook to play with manufactured solutions for Elasticity.

In [1]:
from __future__ import division
from sympy import *
x, y, z, t = symbols('x y z t')
f, g, h = symbols('f g h', cls=Function)
init_printing()

In [2]:
L = symbols('L')
a1, a2, a3, b1, b2, b3, c1, c2, c3 = symbols('a1 a2 a3 b1 b2 b3 c1 c2 c3')
u0, ux, uy, uz, v0, vx, vy, vz, w0, wx, wy, wz = symbols('u0 ux uy uz v0 vx vy vz w0 wx wy wz')
lamda, mu, rho = symbols('lamda mu rho')

In [3]:
u = u0 + ux*sin(a1*pi*x/L) + uy*sin(a2*pi*y/L) + uz*sin(a3*pi*z/L)
v = v0 + vx*sin(b1*pi*x/L) + vy*sin(b2*pi*y/L) + vz*sin(b3*pi*z/L)
w = w0 + wx*sin(c1*pi*x/L) + wy*sin(c2*pi*y/L) + wz*sin(c3*pi*z/L)

In [4]:
def laplacian(U, X=[x,y,z]):
    lap_U = Matrix([sum([diff(U[k], X[j], 2) for j in range(3)]) for k in range(3)])
    return lap_U  


def div(U, X=[x,y,z]):
    return sum([diff(U[k], X[k]) for k in range(3)])


def grad(f, X=[x,y,z]):
    return Matrix([diff(f, X[k]) for k in range(3)])


def navier(U, X=[x,y,z], lamda=lamda, mu=mu):
    return mu*laplacian(U, X) + (lamda + mu)*grad(div(U, X), X)


def dt(U, t=t):
    return Matrix([diff(U[k], t) for k in range(3)])

In [5]:
U = [u,v,w]
F = rho*dt(U) - navier(U)

In [6]:
F

Matrix([
[-mu*(-pi**2*a1**2*ux*sin(pi*a1*x/L)/L**2 - pi**2*a2**2*uy*sin(pi*a2*y/L)/L**2 - pi**2*a3**2*uz*sin(pi*a3*z/L)/L**2) + pi**2*a1**2*ux*(lamda + mu)*sin(pi*a1*x/L)/L**2],
[-mu*(-pi**2*b1**2*vx*sin(pi*b1*x/L)/L**2 - pi**2*b2**2*vy*sin(pi*b2*y/L)/L**2 - pi**2*b3**2*vz*sin(pi*b3*z/L)/L**2) + pi**2*b2**2*vy*(lamda + mu)*sin(pi*b2*y/L)/L**2],
[-mu*(-pi**2*c1**2*wx*sin(pi*c1*x/L)/L**2 - pi**2*c2**2*wy*sin(pi*c2*y/L)/L**2 - pi**2*c3**2*wz*sin(pi*c3*z/L)/L**2) + pi**2*c3**2*wz*(lamda + mu)*sin(pi*c3*z/L)/L**2]])

## References

- Malaya, Nicholas, et al. "MASA: a library for verification using manufactured and analytical solutions." Engineering with Computers 29.4 (2013): 487-496.

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./styles/custom_barba.css', 'r').read()
    return HTML(styles)
css_styling()